## Homework 2.2 Pandas

1. Скачайте этот ноутбук к себе.
2. Заполните пропущенные ячейки, отвечая на заданные вопросы. Там должен быть код! (если не сказано обратное)
3. Сохраните результат в своём гитхаб репозитории.

#### Полезная литература

- http://pandas.pydata.org/pandas-docs/stable/10min.html
- https://pandas.pydata.org/pandas-docs/stable/indexing.html
- https://pandas.pydata.org/pandas-docs/stable/missing_data.html

В этом задании мы с вами рассмотрим датасет [Adult Data Set](https://archive.ics.uci.edu/ml/datasets/Adult).
Основывается он на данных переписи населения 1994 года в США.

Расшифровка содержимого колонок:

- age: continuous. 
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
- fnlwgt: continuous. sampling weight, more here: SIPP Weighting.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
- education-num: continuous. 
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
- sex: Female, Male. 
- capital-gain: continuous. Income from investment sources, apart from wages/salary.
- capital-loss: continuous. Losses from investment sources, apart from wages/salary.
- hours-per-week: continuous. 
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [40]:
%matplotlib inline
import pandas as pd
import numpy as np
pd.__version__

'0.23.4'

In [19]:
columns='age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income'.split(',')
# df = pd.read_csv('../../data/adult.csv.gz', na_values='?') # можно загрузить из файла или URL
df = pd.read_csv('data/adult.csv', na_values='?')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


_Начало анализа датасета в воркшопе_

9) Каковы средние значения и среднеквадратичные отклонения возраста тех, кто получает более 50K в год (признак salary) и тех, кто получает менее 50K в год?

In [23]:
print(df[(df["income"]==">50K")]["age"].mean())
print(df.loc[(df["income"]==">50K")]["age"].std())

print(df[(df["income"]=="<=50K")]["age"].mean())
print(df[(df["income"]=="<=50K")]["age"].std())
#??????????????????????????????????
#Оказывается, с сайта данные некореектно работали. Скачал и все норм))

44.24984058155847
10.519027719851769
36.78373786407767
14.020088490824829


10) Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? (признак education - Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или Doctorate)

In [35]:
df_more = df[df['income'] == ">50K"]

df_edu = df_more.education.isin(["Bachelors","Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"])

print(len(df_more.index))
print(len(df_edu.index))

if len(df_more.index) > len(df_edu.index):
    print('неа')
else:
    print('да')

7841
7841
да


11) Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак marital-status)? Женатыми считаем тех, у кого marital-status начинается с Married (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.

In [36]:
import re

part_marr = (df.loc[(df["income"]==">50K") & df['marital.status'].str.contains(r'Married*')].shape[0]) / (df.loc[df['marital.status'].str.contains(r'Married*')].shape[0])
part_unmarr = (df.loc[(df["income"]==">50K") & (~df['marital.status'].str.contains(r'Married*'))].shape[0]) / (df.loc[~df['marital.status'].str.contains(r'Married*')].shape[0])
print(part_marr)
print(part_unmarr)
print(part_marr > part_unmarr)

0.43692028280469614
0.0644540363975735
True


12) Посчитайте среднее время работы (hours-per-week) зарабатывающих мало и много (salary) для каждой страны (native-country).

In [37]:
big_salary = df.loc[(df["income"]==">50K")].groupby(['native.country'])['hours.per.week'].mean()
small_salary = df.loc[(df["income"]=="<=50K")].groupby(['native.country'])['hours.per.week'].mean()
salary = pd.concat([big_salary, small_salary], axis=1, sort='True')
salary.columns = ['big_salary', 'small_salary']
salary

,big_salary,small_salary
Cambodia,40.000000,41.416667
Canada,45.641026,37.914634
China,38.900000,37.381818
Columbia,50.000000,38.684211
Cuba,42.440000,37.985714
Dominican-Republic,47.000000,42.338235
Ecuador,48.750000,38.041667
El-Salvador,45.000000,36.030928
England,44.533333,40.483333
France,50.750000,41.058824


13) Постройте сводную таблицу для сравнения уровня образования и разницы между capital.gain и capital.loss (то есть пассивный доход). Сделайте так, чтобы capital.gain - capital.loss создали не больше 10 разных уникальных значений. То есть, gain и loss - это числа в каком-то диапазоне. Это количественные признаки из вещественных чисел. С помощью математических функций, типа log, извлечение корня N-ой степени и округления можно перейти к категориальным признакам.

14) Женщины из каких стран получают в среднем большую зарплату (>50K) чаще.

In [38]:
df.loc[((df["sex"]=="Female") & (df["income"]!="<=50K"))].groupby('native.country').size()\
.reset_index(name='counts').sort_values(by=['counts'], ascending=False)

,native.country,counts
30,United-States,1072
22,Philippines,12
0,Canada,9
7,Germany,7
5,England,6
18,Japan,5
1,China,5
16,Italy,5
28,Taiwan,4
2,Cuba,3


15) Создайте случайную колонку - magic_salary, которую нужно будет вычислить следующим образом: если зарплата небольшая (<50K), тогда случайно выберите число из диапазона [0,50]. Если зарплата выше 50K тогда из диапазона [51, 200]. Посчитайте среднюю зарплату в час для групп людей с одни уровнем образования на основе нашей случайной колонки magic_salary

In [42]:
df_rand = df
df_rand['magic.salary'] = np.random.randint(0, 50, df_rand.shape[0])
df_rand.loc[(df_rand['income'] == ">50K"), 'magic.salary'] = np.random.randint(51, 200, df_rand.loc[(df_rand['income'] == ">50K"), 'magic.salary'].shape[0])
tab = pd.pivot_table(df_rand, values='magic.salary', index='education')
tab.sort_values(by=['magic.salary'], ascending=False)

,magic.salary
education,
Prof-school,100.116319
Doctorate,96.740920
Masters,79.030760
Bachelors,66.146032
Assoc-voc,51.434877
Assoc-acdm,50.596064
Some-college,43.603621
HS-grad,40.637558
12th,32.732102
